# Authentication Endpoints Testing

This notebook provides comprehensive testing for all authentication endpoints in the FastAPI CRUD application.

## Endpoints Covered:
- POST `/auth/signup` - User registration
- POST `/auth/login/access-token` - User login
- POST `/auth/test-token` - Token validation
- POST `/auth/password-recovery` - Request password reset
- POST `/auth/reset-password` - Reset password with token
- POST `/auth/logout` - User logout
- POST `/auth/refresh-token` - Refresh access token

In [5]:
# Import required libraries
import requests
import json
from datetime import datetime
import time
from pprint import pprint

# Configuration
BASE_URL = "http://localhost:8000"
API_V1_STR = "/api/v1"
AUTH_URL = f"{BASE_URL}{API_V1_STR}/auth"

# Test data
test_user = {
    "email": f"test_auth_{int(time.time())}@example.com",
    "password": "TestPassword123!",
    "first_name": "Test",
    "last_name": "User",
}

# Global variables for storing tokens
access_token = None
refresh_token = None

print(f"Testing authentication endpoints at: {AUTH_URL}")
print(f"Test user email: {test_user['email']}")

Testing authentication endpoints at: http://localhost:8000/api/v1/auth
Test user email: test_auth_1753927565@example.com


## 1. User Registration (Signup)

Test user registration functionality.

### 🔧 Important: Schema Fix Applied

I've identified and fixed the Pydantic validation error:
- **Issue**: `UserPublicOutput` schema was missing `model_config = ConfigDict(from_attributes=True)`
- **Fix**: Added the required configuration to properly serialize SQLAlchemy models
- **Action Required**: **Restart your FastAPI server** for the changes to take effect

#### To restart your server:
```bash
# Stop the current server (Ctrl+C if running in terminal)
# Then restart with:
make dev
# or
uv run fastapi dev src/main.py --host 0.0.0.0 --port 8000
```

Once restarted, the signup endpoint should work correctly! 🚀

In [ ]:
# Test user signup with enhanced debugging
def test_signup():
    url = f"{AUTH_URL}/signup"

    print(f"📝 Sending signup request...")
    print(f"URL: {url}")
    print(f"Payload: {json.dumps(test_user, indent=2)}")

    response = requests.post(url, json=test_user)

    print(f"\n📋 Response Details:")
    print(f"Status Code: {response.status_code}")
    print(f"Response Headers: {dict(response.headers)}")
    print(f"Response Content-Type: {response.headers.get('content-type', 'N/A')}")

    # Try to parse JSON regardless of status code
    try:
        response_data = response.json()
        print(f"Response JSON: {json.dumps(response_data, indent=2)}")
    except Exception as e:
        print(f"Could not parse JSON: {e}")
        print(f"Raw response text: {response.text}")

    if response.status_code == 201:
        print("\n✅ Signup successful!")
        return response_data if "response_data" in locals() else None
    else:
        print(f"\n❌ Signup failed with status {response.status_code}!")

        # Enhanced error analysis
        if response.status_code == 400:
            print("🔍 This appears to be a validation error.")
            if "response_data" in locals() and "detail" in response_data:
                if "validation error" in str(response_data["detail"]):
                    print("🐛 Pydantic validation error detected!")
                    print("   This suggests the API is returning the wrong data type.")
                    print("   Expected: UserPublicOutput schema")
                    print("   Received: Raw User model object")
                    print("")
                    print(
                        "🔧 FIXED: Added model_config = ConfigDict(from_attributes=True) to UserPublicOutput"
                    )
                    print("❗ Please restart your FastAPI server and try again!")

        return None


print("🧪 Testing user signup endpoint...")
print(
    "⚠️  If you see a Pydantic validation error, restart your FastAPI server after the schema fix!"
)
signup_result = test_signup()

🧪 Testing user signup endpoint...
⚠️  If you see a Pydantic validation error, restart your FastAPI server after the schema fix!
📝 Sending signup request...
URL: http://localhost:8000/api/v1/auth/signup
Payload: {
  "email": "test_auth_1753927565@example.com",
  "password": "TestPassword123!",
  "first_name": "Test",
  "last_name": "User"
}

📋 Response Details:
Status Code: 201
Response Headers: {'date': 'Thu, 31 Jul 2025 02:06:15 GMT', 'server': 'uvicorn', 'content-length': '335', 'content-type': 'application/json'}
Response Content-Type: application/json
Response JSON: {
  "user": {
    "email": "test_auth_1753927565@example.com",
    "first_name": "Test",
    "last_name": "User",
    "is_active": true,
    "id": "3561f7b9-eaba-4e42-ab79-f0eb33ade2fc",
    "is_superuser": false,
    "created_at": "2025-07-31T02:06:16.274335",
    "updated_at": "2025-07-31T02:06:16.274339"
  },
  "message": "Account created successfully",
  "email_verification_required": false
}

✅ Signup successful!



## 2. User Login

Test user authentication and token generation.

In [7]:
# Test user login
def test_login():
    global access_token, refresh_token

    url = f"{AUTH_URL}/login/access-token"

    # Form data for OAuth2 password flow
    login_data = {"username": test_user["email"], "password": test_user["password"]}

    response = requests.post(url, data=login_data)

    print(f"POST {url}")
    print(f"Status Code: {response.status_code}")

    if response.status_code == 200:
        print("✅ Login successful!")
        data = response.json()
        access_token = data.get("access_token")
        refresh_token = data.get("refresh_token")

        print(f"Access Token: {access_token[:50]}...")
        print(f"Refresh Token: {refresh_token[:50] if refresh_token else 'None'}...")
        print(f"Token Type: {data.get('token_type')}")

        return data
    else:
        print("❌ Login failed!")
        print(f"Error: {response.text}")
        return None


login_result = test_login()

POST http://localhost:8000/api/v1/auth/login/access-token
Status Code: 200
✅ Login successful!
Access Token: eyJhbGciOiJIUzI1NiIsInR5cCI6IkpXVCJ9.eyJleHAiOjE3N...
Refresh Token: None...
Token Type: bearer


## 3. Token Validation

Test token validation endpoint.

In [8]:
# Test token validation
def test_token_validation():
    if not access_token:
        print("❌ No access token available. Please run login first.")
        return None

    url = f"{AUTH_URL}/test-token"
    headers = {"Authorization": f"Bearer {access_token}"}

    response = requests.post(url, headers=headers)

    print(f"POST {url}")
    print(f"Status Code: {response.status_code}")

    if response.status_code == 200:
        print("✅ Token is valid!")
        data = response.json()
        pprint(data)
        return data
    else:
        print("❌ Token validation failed!")
        print(f"Error: {response.text}")
        return None


token_validation_result = test_token_validation()

POST http://localhost:8000/api/v1/auth/test-token
Status Code: 200
✅ Token is valid!
{'created_at': '2025-07-31T02:06:16.274335',
 'email': 'test_auth_1753927565@example.com',
 'first_name': 'Test',
 'id': '3561f7b9-eaba-4e42-ab79-f0eb33ade2fc',
 'is_active': True,
 'is_superuser': False,
 'last_name': 'User',
 'updated_at': '2025-07-31T02:06:16.274339'}


## 4. Password Recovery Request

Test password recovery request functionality.

In [9]:
# Test password recovery request
def test_password_recovery():
    url = f"{AUTH_URL}/password-recovery"

    data = {"email": test_user["email"]}

    response = requests.post(url, json=data)

    print(f"POST {url}")
    print(f"Status Code: {response.status_code}")

    if response.status_code == 200:
        print("✅ Password recovery request sent!")
        data = response.json()
        pprint(data)
        return data
    else:
        print("❌ Password recovery request failed!")
        print(f"Error: {response.text}")
        return None


password_recovery_result = test_password_recovery()

POST http://localhost:8000/api/v1/auth/password-recovery
Status Code: 200
✅ Password recovery request sent!
{'expires_in': 60, 'message': 'If the email exists, a reset link has been sent'}


## 5. Token Refresh

Test token refresh functionality.

In [14]:
# Test token refresh
def test_token_refresh():
    if not refresh_token:
        print("❌ No refresh token available. Please run login first.")
        return None

    url = f"{AUTH_URL}/refresh-token"

    data = {"refresh_token": refresh_token}

    response = requests.post(url, json=data)

    print(f"POST {url}")
    print(f"Status Code: {response.status_code}")

    if response.status_code == 200:
        print("✅ Token refresh successful!")
        data = response.json()
        # Update tokens
        global access_token
        access_token = data.get("access_token")
        print(f"New Access Token: {access_token[:50]}...")
        return data
    else:
        print("❌ Token refresh failed!")
        print(f"Error: {response.text}")
        return None


token_refresh_result = test_token_refresh()

❌ No refresh token available. Please run login first.


## 6. User Logout

Test user logout functionality.

In [13]:
# Test user logout
def test_logout():
    global access_token, refresh_token  # Move global to top

    if not access_token:
        print("❌ No access token available. Please run login first.")
        return None

    url = f"{AUTH_URL}/logout"
    headers = {"Authorization": f"Bearer {access_token}"}

    response = requests.post(url, headers=headers)

    print(f"POST {url}")
    print(f"Status Code: {response.status_code}")

    try:
        data = response.json()
    except Exception as e:
        print(f"Could not parse JSON: {e}")
        print(f"Raw response text: {response.text}")
        data = None

    if response.status_code == 200:
        print("✅ Logout successful!")
        pprint(data)
        access_token = None
        refresh_token = None
        return data
    else:
        print("❌ Logout failed!")
        pprint(data)
        return None

logout_result = test_logout()

POST http://localhost:8000/api/v1/auth/logout
Status Code: 422
❌ Logout failed!
{'detail': [{'input': None,
             'loc': ['body'],
             'msg': 'Field required',
             'type': 'missing'}]}


In [21]:
# Fixed logout test function with proper JSON body
def test_logout_fixed():
    global access_token, refresh_token

    if not access_token:
        print("❌ No access token available. Please run login first.")
        return None

    url = f"{AUTH_URL}/logout"
    headers = {"Authorization": f"Bearer {access_token}"}
    
    # FIXED: Send proper JSON body with LogoutRequest schema
    logout_data = {
        "all_devices": False  # Set to True to logout from all devices
    }

    response = requests.post(url, headers=headers, json=logout_data)

    print(f"POST {url}")
    print(f"Request Body: {logout_data}")
    print(f"Status Code: {response.status_code}")

    try:
        data = response.json()
    except Exception as e:
        print(f"Could not parse JSON: {e}")
        print(f"Raw response text: {response.text}")
        data = None

    if response.status_code == 200:
        print("✅ Logout successful!")
        print(f"Response: {data}")
        access_token = None
        refresh_token = None
        return data
    else:
        print("❌ Logout failed!")
        print(f"Error: {data}")
        return None


#test_logout_fixed()
# In your notebook, load the fixed functions:
exec(open('fixed_logout_tests.py').read())

🔧 Fixed logout functions created!
📝 Use test_logout_fixed() and run_complete_auth_test_suite_fixed() for proper testing


## 7. Error Cases Testing

Test various error scenarios.

In [15]:
# Test error cases
def test_error_cases():
    print("Testing error cases...\n")

    # 1. Login with invalid credentials
    print("1. Testing login with invalid credentials:")
    invalid_login = {"username": "invalid@example.com", "password": "wrongpassword"}
    response = requests.post(f"{AUTH_URL}/login/access-token", data=invalid_login)
    print(f"Status: {response.status_code}, Response: {response.text}\n")

    # 2. Access protected endpoint without token
    print("2. Testing protected endpoint without token:")
    response = requests.post(f"{AUTH_URL}/test-token")
    print(f"Status: {response.status_code}, Response: {response.text}\n")

    # 3. Access protected endpoint with invalid token
    print("3. Testing protected endpoint with invalid token:")
    headers = {"Authorization": "Bearer invalid_token"}
    response = requests.post(f"{AUTH_URL}/test-token", headers=headers)
    print(f"Status: {response.status_code}, Response: {response.text}\n")

    # 4. Duplicate signup
    print("4. Testing duplicate signup:")
    response = requests.post(f"{AUTH_URL}/signup", json=test_user)
    print(f"Status: {response.status_code}, Response: {response.text}\n")


test_error_cases()

Testing error cases...

1. Testing login with invalid credentials:
Status: 400, Response: {"detail":"Incorrect email or password"}

2. Testing protected endpoint without token:
Status: 401, Response: {"detail":"Not authenticated"}

3. Testing protected endpoint with invalid token:
Status: 403, Response: {"detail":"Could not validate credentials"}

4. Testing duplicate signup:
Status: 400, Response: {"detail":"User with email test_auth_1753927565@example.com already exists"}



## 8. Complete Test Summary

Run all tests and provide a summary.

In [16]:
# Complete test suite
def run_complete_auth_test_suite():
    print("🧪 Running Complete Authentication Test Suite")
    print("=" * 50)

    results = {"timestamp": datetime.now().isoformat(), "tests": {}}

    # Create new test user for complete suite
    suite_user = {
        "email": f"suite_test_{int(time.time())}@example.com",
        "password": "SuiteTestPassword123!",
        "first_name": "Suite",
        "last_name": "Test",
    }

    # 1. Signup
    print("\n1. Testing Signup...")
    signup_response = requests.post(f"{AUTH_URL}/signup", json=suite_user)
    results["tests"]["signup"] = {
        "status_code": signup_response.status_code,
        "success": signup_response.status_code == 201,
    }
    print(
        f"   Result: {'✅ PASS' if results['tests']['signup']['success'] else '❌ FAIL'}"
    )

    # 2. Login
    print("\n2. Testing Login...")
    login_data = {"username": suite_user["email"], "password": suite_user["password"]}
    login_response = requests.post(f"{AUTH_URL}/login/access-token", data=login_data)
    results["tests"]["login"] = {
        "status_code": login_response.status_code,
        "success": login_response.status_code == 200,
    }
    print(
        f"   Result: {'✅ PASS' if results['tests']['login']['success'] else '❌ FAIL'}"
    )

    # Get token for subsequent tests
    if results["tests"]["login"]["success"]:
        token_data = login_response.json()
        suite_token = token_data.get("access_token")
        headers = {"Authorization": f"Bearer {suite_token}"}

        # 3. Token validation
        print("\n3. Testing Token Validation...")
        token_response = requests.post(f"{AUTH_URL}/test-token", headers=headers)
        results["tests"]["token_validation"] = {
            "status_code": token_response.status_code,
            "success": token_response.status_code == 200,
        }
        print(
            f"   Result: {'✅ PASS' if results['tests']['token_validation']['success'] else '❌ FAIL'}"
        )

        # 4. Logout
        print("\n4. Testing Logout...")
        logout_response = requests.post(f"{AUTH_URL}/logout", headers=headers)
        results["tests"]["logout"] = {
            "status_code": logout_response.status_code,
            "success": logout_response.status_code == 200,
        }
        print(
            f"   Result: {'✅ PASS' if results['tests']['logout']['success'] else '❌ FAIL'}"
        )

    # Summary
    print("\n" + "=" * 50)
    print("📊 TEST SUMMARY")
    print("=" * 50)

    total_tests = len(results["tests"])
    passed_tests = sum(1 for test in results["tests"].values() if test["success"])

    print(f"Total Tests: {total_tests}")
    print(f"Passed: {passed_tests}")
    print(f"Failed: {total_tests - passed_tests}")
    print(f"Success Rate: {(passed_tests/total_tests)*100:.1f}%")

    return results


test_results = run_complete_auth_test_suite()

🧪 Running Complete Authentication Test Suite

1. Testing Signup...
   Result: ✅ PASS

2. Testing Login...
   Result: ✅ PASS

3. Testing Token Validation...
   Result: ✅ PASS

4. Testing Logout...
   Result: ❌ FAIL

📊 TEST SUMMARY
Total Tests: 4
Passed: 3
Failed: 1
Success Rate: 75.0%
   Result: ✅ PASS

2. Testing Login...
   Result: ✅ PASS

3. Testing Token Validation...
   Result: ✅ PASS

4. Testing Logout...
   Result: ❌ FAIL

📊 TEST SUMMARY
Total Tests: 4
Passed: 3
Failed: 1
Success Rate: 75.0%


In [19]:
# Also fix the complete test suite logout test
def run_complete_auth_test_suite_fixed():
    print("🧪 Running Complete Authentication Test Suite (FIXED)")
    print("=" * 50)

    results = {"timestamp": datetime.now().isoformat(), "tests": {}}

    # Create new test user for complete suite
    suite_user = {
        "email": f"suite_test_{int(time.time())}@example.com",
        "password": "SuiteTestPassword123!",
        "first_name": "Suite",
        "last_name": "Test",
    }

    # 1. Signup
    print("\n1. Testing Signup...")
    signup_response = requests.post(f"{AUTH_URL}/signup", json=suite_user)
    results["tests"]["signup"] = {
        "status_code": signup_response.status_code,
        "success": signup_response.status_code == 201,
    }
    print(f"   Result: {'✅ PASS' if results['tests']['signup']['success'] else '❌ FAIL'}")

    # 2. Login
    print("\n2. Testing Login...")
    login_data = {"username": suite_user["email"], "password": suite_user["password"]}
    login_response = requests.post(f"{AUTH_URL}/login/access-token", data=login_data)
    results["tests"]["login"] = {
        "status_code": login_response.status_code,
        "success": login_response.status_code == 200,
    }
    print(f"   Result: {'✅ PASS' if results['tests']['login']['success'] else '❌ FAIL'}")

    # Get token for subsequent tests
    if results["tests"]["login"]["success"]:
        token_data = login_response.json()
        suite_token = token_data.get("access_token")
        headers = {"Authorization": f"Bearer {suite_token}"}

        # 3. Token validation
        print("\n3. Testing Token Validation...")
        token_response = requests.post(f"{AUTH_URL}/test-token", headers=headers)
        results["tests"]["token_validation"] = {
            "status_code": token_response.status_code,
            "success": token_response.status_code == 200,
        }
        print(f"   Result: {'✅ PASS' if results['tests']['token_validation']['success'] else '❌ FAIL'}")

        # 4. Logout (FIXED with proper JSON body)
        print("\n4. Testing Logout...")
        logout_data = {"all_devices": False}
        logout_response = requests.post(f"{AUTH_URL}/logout", headers=headers, json=logout_data)
        results["tests"]["logout"] = {
            "status_code": logout_response.status_code,
            "success": logout_response.status_code == 200,
        }
        print(f"   Result: {'✅ PASS' if results['tests']['logout']['success'] else '❌ FAIL'}")

    # Summary
    print("\n" + "=" * 50)
    print("📊 TEST SUMMARY")
    print("=" * 50)

    total_tests = len(results["tests"])
    passed_tests = sum(1 for test in results["tests"].values() if test["success"])

    print(f"Total Tests: {total_tests}")
    print(f"Passed: {passed_tests}")
    print(f"Failed: {total_tests - passed_tests}")
    print(f"Success Rate: {(passed_tests/total_tests)*100:.1f}%")

    return results

print("🔧 Fixed logout functions created!")
print("📝 Use test_logout_fixed() and run_complete_auth_test_suite_fixed() for proper testing")


🔧 Fixed logout functions created!
📝 Use test_logout_fixed() and run_complete_auth_test_suite_fixed() for proper testing
